# Vizualization of the results

In this notebook we just visualize and format the pickle file containing experiments results

In [1]:
import pickle as pkl
import pandas as pd

pd.set_option("display.max_rows", 999)

In [5]:
def print_results(filename, metrics=None):
    with open(filename, "rb") as f:
        results = pkl.load(f)

    print("commit number %s: datetime: %s" % (results["commit"], results["datetime"]))

    if metrics is None:
        metrics = ["avg_prec_w", "avg_prec_w_train", "log_loss"]
    
    df = results["results"]


    return df

In [2]:
def print_results(filename, metrics=None):
    with open(filename, "rb") as f:
        results = pkl.load(f)

    print("commit number %s: datetime: %s" % (results["commit"], results["datetime"]))

    if metrics is None:
        metrics = ["avg_prec_w", "avg_prec_w_train", "log_loss"]
    
    df = results["results"]

    return (
        df[["dataset", "classifier_title", *metrics, "repeat"]]
        .groupby(["dataset", "classifier_title", "repeat"])
        # .agg(["mean", "std"])
        .agg(["mean"])
        .reset_index()
        .loc[:, ["dataset", "classifier_title", *metrics]]    
        .pivot(index="dataset", columns="classifier_title")
        .style
        .format("{:.2f}")
        .background_gradient(axis="index")    
    )

In [6]:
filename = "../benchmarks_2021-05-11-16:56:05.pickle"
print_results(filename)

commit number 14fe539: datetime: 2021-05-11-16:56:05


,datasets,classifier,classifier_title,repeat,fit_time,predict_time,roc_auc,roc_auc_w,avg_prec,avg_prec_w,log_loss,accuracy,roc_auc_train,roc_auc_w_train,avg_prec_train,avg_prec_w_train,log_loss_train,accuracy_train
0,adult,ForestClassifier,WildWood (One Hot),0,5.206055,0.151765,0.913282,0.913282,0.799660,0.799660,0.301078,0.862554,0.919141,0.919141,0.810751,0.810751,0.293244,0.865216
1,bank,ForestClassifier,WildWood (One Hot),0,0.756556,0.131515,0.907173,0.907173,0.557236,0.557236,0.225048,0.902389,0.923850,0.923850,0.614138,0.614138,0.208610,0.907732
2,churn,ForestClassifier,WildWood (One Hot),0,0.056175,0.010105,0.877996,0.877996,0.734180,0.734180,0.239046,0.931000,0.929322,0.929322,0.818636,0.818636,0.194503,0.942992
3,default-cb,ForestClassifier,WildWood (One Hot),0,0.455634,0.078373,0.765032,0.765032,0.526870,0.526870,0.439868,0.817333,0.789463,0.789463,0.569878,0.569878,0.424322,0.824190
4,adult,ForestClassifier,WildWood (Ordinal),0,0.250342,0.102136,0.911116,0.911116,0.790849,0.790849,0.303958,0.857094,0.916073,0.916073,0.804393,0.804393,0.298035,0.862554
5,bank,ForestClassifier,WildWood (Ordinal),0,0.195421,0.088195,0.905198,0.905198,0.554132,0.554132,0.225803,0.900472,0.923508,0.923508,0.606661,0.606661,0.207934,0.905236
6,churn,ForestClassifier,WildWood (Ordinal),0,0.021979,0.006385,0.881363,0.881363,0.761887,0.761887,0.228402,0.925000,0.938617,0.938617,0.843919,0.843919,0.178717,0.942992
7,default-cb,ForestClassifier,WildWood (Ordinal),0,0.369877,0.074827,0.764674,0.764674,0.522417,0.522417,0.439652,0.818111,0.789276,0.789276,0.566545,0.566545,0.424219,0.824333
8,adult,ForestClassifier,WildWood (Categorical),0,0.253332,0.110918,0.908994,0.908994,0.780025,0.780025,0.308654,0.856275,0.916425,0.916425,0.794382,0.794382,0.298381,0.862671
9,bank,ForestClassifier,WildWood (Categorical),0,0.196035,0.095903,0.896433,0.896433,0.547222,0.547222,0.233203,0.899882,0.921757,0.921757,0.605914,0.605914,0.210465,0.906310


## Resultats n_estimators=100, sans aggregation et max_features=None, dirichlet=0.0

In [17]:
filename = "../benchmarks_2021-04-19-15:25:42.pickle"

print_results(filename)

commit number 6ad6dfc: datetime: 2021-04-19-15:25:42


## Resultats n_estimators=100, sans aggregation et max_features="auto", dirichlet=0.0

In [20]:
filename = "../benchmarks_2021-04-19-15:29:20.pickle"
print_results(filename, metrics=["avg_prec_w", "fit_time"])

commit number 6ad6dfc: datetime: 2021-04-19-15:29:20


## Resultats n_estimators=100, sans aggregation et max_features="auto", dirichlet=0.5

In [21]:
filename = "../benchmarks_2021-04-19-15:33:27.pickle"
print_results(filename, metrics=["avg_prec_w", "fit_time"])

commit number 6ad6dfc: datetime: 2021-04-19-15:33:27


## Resultats n_estimators=100, avec aggregation et max_features="auto", dirichlet=0.5

In [22]:
filename = "../benchmarks_2021-04-19-15:37:09.pickle"
print_results(filename, metrics=["avg_prec_w", "fit_time"])

commit number 6ad6dfc: datetime: 2021-04-19-15:37:09


In [ ]:
import sys
import subprocess
from time import time
from datetime import datetime
import logging
import pickle as pkl
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    log_loss,
    accuracy_score,
)
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier

sys.path.extend([".", ".."])

from wildwood.dataset import loaders_small_classification, load_churn
from wildwood.forest import ForestClassifier


from wildwood.dataset import (
    load_adult,
    load_bank,
    load_breastcancer,
    load_car,
    load_cardio,
    load_churn,
    load_default_cb,
    load_letter,
    load_satimage,
    load_sensorless,
    load_spambase,
)

In [ ]:
dataset = load_bank()

In [ ]:
clf = ForestClassifier(
    n_estimators=1,
    n_jobs=1,
    class_weight="balanced",
    random_state=42,
    aggregation=False,
    max_features=None,
    dirichlet=0.0
)

In [ ]:
dataset.one_hot_encode = True
dataset.standardize = False
dataset.drop = None
X_train, X_test, y_train, y_test = dataset.extract(random_state=42)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
from bokeh.plotting import show, output_notebook
from wildwood.plot import plot_tree

output_notebook()

In [ ]:
fig = plot_tree(clf, height=900, width=900)

In [ ]:
show(fig)

In [ ]:
dataset.one_hot_encode = False
dataset.standardize = False
dataset.drop = None
X_train, X_test, y_train, y_test = dataset.extract(random_state=42)

In [ ]:
clf = ForestClassifier(
    n_estimators=1,
    n_jobs=1,
    class_weight="balanced",
    random_state=42,
    aggregation=False,
    max_features=None,
    dirichlet=0.0,
    categorical_features=dataset.categorical_features_
)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
fig = plot_tree(clf, height=900, width=900)

In [ ]:
show(fig)

In [ ]:
df = clf.get_nodes(0)

In [ ]:
pd.set_option("display.max_columns", 99)

In [ ]:
df.loc[338:342]

In [ ]:
clf.trees[0]._tree.

In [ ]:
partition_train = clf.trees[0]._tree_context.partition_train[19512:19517]

In [ ]:
partition_valid = clf.trees[0]._tree_context.partition_valid[11271:11273]

In [ ]:
pd.DataFrame(X_train[partition_train, :])

In [ ]:
pd.DataFrame(X_train[partition_valid, :])

In [ ]:
clf.trees[0]._tree_context.partition_train